In [1]:
From Coq Require Import ZArith Reals Psatz.
From Flocq Require Import Core Plus_error.

Open Scope R_scope.

Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

## The Leapfrog Integration Scheme

The "leapfrog" integrator is commonly used in scientific computing for modeling the behavior of a physical system over a long period of time starting from some initial conditions. 

For a one-dimensional system, the equations of motion are 
$$\dot{x} = v$$
$$m\dot{v} = F(x) = -\frac{dU(x)}{dx},$$
where $x$ is position of the body under consideration with mass $m$ (e.g. a planet, particle, etc.), $v$ is the velocity, $F(x)$ is the force on the body when it is at position $x$, and $U(x)$ is the potential energy of the body. 

One step of the "leapfrog" (aka velocity [Verlet](https://en.wikipedia.org/wiki/Verlet_integration)) integrator for predicting the future position and velocity $(x_{n+1}, v_{n+1})$ of the above equations of motion starting from $(x_n,v_n)$ is given by 

$$v_{n+1/2} = v_n + \frac{1}{2}hF(x_n)$$

$$x_{n+1} = x_n + hv_{n+1/2}$$

$$v_{n+1} = v_{n+1/2} + \frac{1}{2} h F(x_{n+1}),$$

where $h$ is the *time step* indicating how far forward in time the system has advanced. 

We can define the leapfrog integrator for a body with unit mass as follows. 

In [130]:
Definition leapfrog_12 xn vn h F := 
    let vnp12 := vn + 0.5*  h* F xn in
    let xnp1  := xn + h* vnp12 in
    pair xnp1 vnp12.
    
Definition leapfrog_step x0 v0 h F := 
    let v12 := snd (leapfrog_12 x0 v0 h F) in
    let x1  := fst (leapfrog_12 x0 v0 h F) in
    let v1  := v12 + 0.5* h* F x1 in
    pair x1 v1.
    
Fixpoint leapfrog x0 v0 h F (n:nat):=
    match n with 
    | 0 => pair x0 v0
    | S n' => 
        let xn := fst (leapfrog_step x0 v0 h F) in
        let vn := snd (leapfrog_step x0 v0 h F) in
        leapfrog xn vn h F (n')
    end.

Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

## Conservation of momentum and inexact trajectories

Let's analyze the leapfrog scheme using the simple harmonic oscillator as an example. 

The potential energy of the simple harmonic oscillator is 
$$U(x) = \frac{1}{2}kx^2.$$ Observe that the forcing function in this case is linear: $$F(x) = -kx.$$

The kinetic energy is given by 
$$K(x) = \frac{1}{2}mv^2,$$

so the total energy of the system is $$E = U(x) + K(x),$$

and the exact solutions to the equations of motion with $m = k =1$ and initial position and velocity $(x_0,v_0)$ are 

$$x(t)       = x_0\cos (\omega t) - v_0\sin (\omega t)$$ 
$$v(t)       = x_0\sin (\omega t) + v_0\cos (\omega t),$$ 

where $\omega = \sqrt{k/m}.$

Given these exact solutions, we can see that the total energy of the system is $E=\frac{1}{2}$; we would like to see that our integrator conserves the total energy of the system. 

Suppose that $k = \pi^2$

In [133]:
Definition F (x:R) := -1 * x.

Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

In [134]:
Lemma leapfrog_fixpoint :  forall x0 v0 h F (n:nat), leapfrog x0 v0 h F (n:nat) =
    match n with 
    | 0 => pair x0 v0
    | S n' => 
        let xn := fst (leapfrog_step x0 v0 h F) in
        let vn := snd (leapfrog_step x0 v0 h F) in
        leapfrog xn vn h F (n')
    end.
Proof.
intros. induction n. 
simpl. auto.
simpl. auto. 
Qed.

Cell evaluated. 
 

 
 
 Cell rolled back. 
 

 
 
 
 Rollback cell 
 

 
 
 Auto rollback

In [7]:
Lemma leapfrog_returns_1:
    forall x0 v0 h,
    x0 = 1 -> v0 = 0 -> h = 2 ->
    (leapfrog x0 v0 h F 2) = pair x0 v0. 

Proof.
intros x0 v0 h Hx Hv Hh.
unfold leapfrog. unfold F.
simpl; replace x0 with 1; replace v0 with 0; replace h with 2. 
replace (1 + 2 * (0 + 0.5 * 2 * (-1 * 1)) +
 2 *
 (0 + 0.5 * 2 * (-1 * 1) +
  0.5 * 2 * (-1 * (1 + 2 * (0 + 0.5 * 2 * (-1 * 1)))) +
  0.5 * 2 * (-1 * (1 + 2 * (0 + 0.5 * 2 * (-1 * 1)))))) with 1 by nra;
replace (0 + 0.5 * 2 * (-1 * 1) + 0.5 * 2 * (-1 * (1 + 2 * (0 + 0.5 * 2 * (-1 * 1)))) +
0.5 * 2 * (-1 * (1 + 2 * (0 + 0.5 * 2 * (-1 * 1)))) + 
0.5 * 2 * (-1 * 1)) with 0 by nra. 
auto with real.
Qed.

Cell rolled back.

In [200]:
Lemma leapfrog_returns_2:
    forall x0 v0 h,
    x0 = 1 -> v0 = 0 -> h = (0.2 * 2 * PI) ->
    (leapfrog x0 v0 h F 5) = pair x0 v0. 
    
Proof.
intros x0 v0 h Hx Hv Hh.
functional induction.

Error: The reference functional was not found in the current environment.

The following definition of a symplectic map is taken from the paper [Why is the Boris alogrithm so good?](https://aip-scitation-org.proxy.library.cornell.edu/doi/pdf/10.1063/1.4818428), which clearly communicates the fact that symplectic maps are always volume preserving, but volume preserving maps are not necessarily symplectic. 

A one-step map $$\psi : \mathbf{z}_k \equiv (\mathbf{x}_k, \mathbf{v}_k) \rightarrow \mathbf{z}_{k+1} \equiv (\mathbf{x}_{k+1}, \mathbf{v}_{k+1})$$ is **symplectic** if its [Jacobian matrix](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) $$A = \frac{\partial \psi }{\partial \mathbf{z}_k}$$ is **symplectic**. Specifically, if $$A^T J A = J$$ where $$ J =  \begin{pmatrix}
0 & I \\
-I & 0 
\end{pmatrix}.$$ 

For a system of dimension $n$, we can write the block Jacobian with blocks of size $n \times n$, 

$$ A =  \begin{pmatrix}
S_1 & S_2 \\
S_3 & S_4
\end{pmatrix}.$$ 

We can therefore define a **symplectic** Jacobian as one with blocks that satisfy

$$S_1S_2^T = S_2S_1^T $$

$$S_3S_4^T = S_4S_3^T $$

$$S_1S_4^T - S_2S_3^T = I.$$

Given that we are working with at most three-dimensions, the above set of necessary and sufficient conditions for symplecticity on the block componenets of the Jacobian allow us to avoid dealing with more abstract definitions of matrices in Coq.

In the one-dimensional case, observe that area-preservation is a sufficient condition for symplecticity. More specifically, the determinant of the Jacobian of a one-dimensional symplectic map is 1. 

In [ ]:
Definition is_symplectic_1d (jacobian:(R *R)*(R *R)) := 
    let S1 := fst(fst(jacobian)) in
    let S2 := snd(fst(jacobian)) in
    let S3 := fst(snd(jacobian)) in
    let S4 := snd(snd(jacobian)) in 
    S1*S4 - S2*S3 = 1.

In [ ]:
Definition jacob_leapfrog_1 xn h (dFdx: R -> R) :=
    let s1 := 1.0 in
    let s2 := h in
    let s3 := h* dFdx xn in 
    let s4 := 1.0 in 
    pair (pair s1 s2) (pair s3 s4). 

In [ ]:
Lemma leapfrog1_is_symplectic:
     forall x0 h,
     x0 = 0 -> Rgt h 0 ->
     is_symplectic_1d (jacob_leapfrog_1 x0 h id).
     
Proof. 
intros x0 h Hx Hh; unfold is_symplectic_1d; unfold jacob_leapfrog_1; unfold fst; unfold snd; unfold id; nra. 